# IBM Qiskit Primitives v2

Sampler primitives
    - low-level execution
    - returns single shot measurement
    - circuit should include measurement

Estimator primitives
    - high-level execution
    - returns expectation values of observables
    - circuit should not include measurement

PUB - Primitive Unified Bloc
    - program input for the Sampler
    - generally a tuple

    - Circuit - ISA quantum circuit containing 1 or more ClassicalRegister and measure instructions
    - Parameter Values - tensor (ND-array) of sets of parameter values to evaluate parametric circuit

Shots - number of samples or repetitions to measure the circuit

Allowed pub-like inputs

In [1]:
# Load qiskit runtime service and select backend
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.backend('ibm_sherbrooke')

/var/folders/kf/vmdscp_n18qfgr6xz5zv0sw40000gn/T/ipykernel_41205/230416974.py:4: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud", "ibm_quantum_platform", and "local" will be the only valid channels. Open Plan users should migrate now.  All other users should review the migration guide (https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp)to learn when to migrate.
  service = QiskitRuntimeService()


In [2]:
# Create a circuit and transpile to a backend ISA circuit
from qiskit import QuantumCircuit, transpile

bell_meas = QuantumCircuit(2)
bell_meas.h(0)
bell_meas.cx(0, 1)
bell_meas.measure_all()

isa_bell_meas = transpile(bell_meas, backend)
pub_bell_meas = (isa_bell_meas)

In [ ]:
# Initialize a Sampler primitive and run pub
from qiskit_ibm_runtime import SamplerV2

sampler = SamplerV2(backend)
job_bell_meas = sampler.run([pub_bell_meas], shots=2)

result_bell_meas = job_bell_meas.result()
data_bell_meas = result_bell_meas[0].data
# data_bell_meas
print(data_bell_meas)
## this block of code does not stop running - neither at shots=10 nor at shots=2
## it went as long as two minutes before I stopped it

Sampler job returns `PrimitiveResult` object containing ordered list of PubResult 
PubResult = data container for a single input Pub's execution result
PubResult.data = contains measurement outcome data for classical registers in pub circuit
PubResult.metadata = metadata a primitive might record

PubResult.data = DataBin class
DataBin stores outcomes of measurements in ClassicalRegisters in the pub circuit



In [ ]:
# examples of accessing outcomes
bits = data_bell_meas.meas
print("shape: ", bits.shape)
print("num_bits: ", bits.num_bits)
print("num_shots: ", bits.num_shots)
print("array \n", bits.array)
## this block does not run because in the previous block data_bell_meas is never defined due to the long code run

## ISA Circuits

ISA circuit is a QuantumCircuit satisfying:
1. same number of qubits as device
2. only contains basic gates for a device
3. satisfies connectivity of a device

Obtained by transpilation

## Parametric circuits

* circuit that contains unbound parameter


In [ ]:
# SamplerV2 parametric example
## This is an example only because this exercise does not define para_bell_meas

import numpy as np

# parameter values to evaluate with 20 theta values
param_vals = np.linspace(0, np.pi, 20)

# transpile to an ISA circuit for the intended backend
isa_par_bell_meas = transpile(par_bell_meas, backend)

# construct pub and run
# pub and result shape
pub_par_bell_meas = (isa_par_bell_meas, param_vals)
job_par_bell_meas = sampler.run([pub_par_bell_meas], shots=1000)
result_par_bell_meas = job_par_bell_meas.result()

# extract creg data
bits = result_par_bell_meas[0].data.meas

########

import matplotlib.pyplot as plt
import seaborn
seaborn.set()

p0s = np.sum(bits.array == 0, axis=1) / bits.num_shots
p1s = np.sum(bits.array == 3, axis=1) / bits.num_shots

plt.plot(param_vals, p0s, label="P('00')")
plt.plot(param_vals, p1s, label="P('11')")
plt.xlabel("0")
plt.legend()
plt.show()

2025-06-29 - STOPPED HERE - 14:46 IN THE VIDEO - I need to cut off this exercise until later, maybe tomorrow

2025-07-01, Tue - started again

In [ ]:
# Sampler V2 shots

#1 all shots in pub
pub1a = (isa_bell_meas, None, 100)
pub1b = (isa_bell_meas, None, 200)

result1 = sampler.run([pub1a, pub1b], shots=1000).result()

for i, res in enumerate(result1):
    print(f"Pub {i}: num_shots = {res.data.meas.num_shots}")
    

In [ ]:
#2 fall back to 'run' shots
pub2a = (isa_bell_meas)
pub2b = (isa_bell_meas, None, 200)

result2 = sampler.run([pub2a, pub2b], shots=1000).result()

for i, res in enumerate(result2):
    print(f"Pub {i}: num_shots={res.data.meas.num_shots}")

In [ ]:
#3 fallback to 'default' shots
pub3a = (isa_bell_meas)
pub3b = (isa_bell_meas, None, 200)

result3 = sampler.run([pub3a, pub3b]).result()

for i, res in enumerate(result3):
    print(f"Pub {i}: num_shots = {res.data.meas.num_shots}")

## Estimator V2
Estimator primitives are higher-level than Sampler V2

- defined by its run method

```python
    Estimator.run(
        pubs: list[EstimatorPub],
        precision: float | None = None,
    ) -> Job[EstimatorResult]
```

desired precision is optional

In [ ]:
# representing observables
# create isa circuit

from qiskit_ibm_runtime import EstimatorV2

# bell circuit
bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0,1)

# transpile
isa_bell = transpile(bell, backend)

In [ ]:
# create isa observable
import qiskit.quantum_info as qi

obs = qi.SparsePauliOp(["ZZ"])

# transpile
isa_obs = obs.apply_layout(isa_bell.layout)

In [ ]:
# estimator databin

# construct estimator
estimator = EstimatorV2(backend)

# construct pub and run
pub = (isa_bell, isa_obs)
est_result_bell = estimator.run([pub])

# view result data
data = est_result_bell[0].data
evs = data.evs
stds = data.stds
print(f"<ZZ> = {evs: .3f} +/- {stds: 3f}")

## Estimator V2 shape

-Trivial shaped pubs hava shape == ()
-These return a single floast as the evs result

Examples of shape==() pubs
- a non-parametric pub with a single observable
- a parametric pub with a single parameter value and a single observable

## Estimator V2 Broadcating

ND-array broadcasting rules
- input arrays do not need to have the same number of dimensions
- the resulting array will have the same number of dimensions as the largest
- the size of each dimension is the largest size of corresponding dimension
- missing dimensions are assumed to have size one
- it starts with the trailing (rightmost) dimension and works its way left
- two dimensions are compatible when their sizes are equal or one of them is 1

Example:
- A         (2d array):     5 x 4
- B         (1d array):         1
- Result    (2d array):     5 x 4

Example:
- A         (3d array):     15 x 3 x 5
- B         (3d array):     15 x 1 x 5
- Result    (3d array):     15 x 3 x 5

In [ ]:
# parameterized bell circuit
theta = Parameter("0")  # is this supposed to be theta symbol, not zero?
par_bell = QuantumCircuit(2)
par_bell.ry(theta, 0)
par_bell.cx(0,1)

# transpile to circuit
isa_par_bell = transpile(par_bell, backend)

# parameter values to evaluate with 20 theta values
param_vals = np.linspace(0, np.pi, 20)

# create observable array with 3 observables
par_bell_obs = [
    qi.SparsePauliOp(["XX"])
    qi.SparsePauliOp(["YY"])
    qi.SparsePauliOp(["ZZ"])
]

# transpile to observable and reshape to (3,1) array
isa_par_bell_obs = [
    [op.apply_layout(isa_par_bell.layout)]
    for op in par_bell_obs
]

# construct pub and run
pub = (isa_par_bell, isa_par_bell_obs, param_vals)

# run estimator 
est_job_par_bell = estimator.run([pub])
est_result_par_bell = est_job_par_bell.result()
evs = est_result_par_bell[0].data.evs
stds = est_result_par_bell[0].data.stds

## ISA Observables

What is an ISA Observable?
1. linear combination of Pauli's with real coefficients
2. each Pauli is defined on the same number of qubits as an ISA circuit

- if you are used to working with abstract observables you will need to apply the layout of a transpile circuit to map the observable qubits
- several `SparsePauliOp` and `Pauli` operator classes have an `apply_layout` method to simplify this process

In [ ]:
# example
abstract_example = QuantumCircuit(1)
abstract_example.h(0)
isa_example = transpile(
    abstract_example, backend, initial_layout=[4]
)
display(isa_example.draw('mpl', idle_wires=False))

abstract_obs = qi.SparsePauliOp(["X", "Y", "Z"])
abstract_obs.apply_layout(isa_example.layout)

## Estimator Containers

- `ObservablesArray` and `BindingsArray` are new helper classes for manipulating these arrays
- `ObservablesArray` is like NumPy array of SparsePauliOp observables
- BindingsArray is like a NumPy array for names parameter value sets
- Have convenience methods like (eg: shape)

In [ ]:
# Example
from qiskit.primitives.containers.observables_array import (ObservablesArray)

# shape (3,1) list of Pauli Array
obs = ObservablesArray(["XX", "YY", "ZZ"]).reshape(3,1)

# apply ISA circuit layout
isa_obs = obs.apply_layout(isa_par_bell.layout)

from qiskit.primitives.containers.bindings_array import (BindingsArray)

BindingsArray({tuple(isa_par_bell.parameters): param_vals})
BindingsArray(<shape=(20,0), num_parameters=1, parameters=['0']>)

## Measurement Observables

- IBM primitives apply Pauli grouping to minimize the number of measurement circuits
- groups Paulis that can be computed from marginals of a single measurement
- grouping is done via the `PauliList.group_commuting(quibit_wise=True)`

```Python
    paulis = qi.PauliList([
        "ZII", "IZI", "IIZ",
        "XII", "IXI", "IIX",
        "YII", "IYI", "IIY" 
    ])
    paulis.group_commuting(qubit_wise=True)
```

## IBM runtime primitives options
- the sampler and estimator API described allows creation of portable programs
- the IBM runtime primitives support additional functionality which can be configured by options
- enables automatic error suppression and mitigation methods

Sampler Options:
- `default_shots` 
- `dynamical_decoupling`
- `twirling`

Estimator options
- `default_precision`
- `default_shots`
- `reilience_level`
- `dynamical_coupling`
- `twirling`
- `resilience`


## Pauli Twirling
- requires transpiling circuit to identify layers of twirled 2-qubit gates or mesaurements
- reparameterizes the circuit to allow parametrically inserting random paulis
- samples randomly paulis to insert into the circuit in a specific way
- averages (estimator) or concatenates (sampler) the results over samples


## Twirling Options
twirling is controlled via the following twirling options

`enable_gates` whether to apply 2-qubit gate twirling
`enable_measure` whether to apply twirling of measurements
`num_randomizations` number of random samples to use when twirling or performing mitigation
`shots_per_randomization` number of shots to run for each random sample
`strategy` specify the strategy of twirling qubits in identified layers of 2-qubit twirled gates

In [ ]:
# set options after initializing
twirl_sampler = SamplerV2(backend)
twirl_sampler.options.twirling.enable_gates = True
twirl_sampler.options.twirling.enable_measure = True
twirl_sampler.options.twirling.strategy = 'active-circuit'

# Pass options during initialization
options = {
    "twirling": {
        "enable_gates": True,
        "enable_Measure": True,
        "strategy": "active-circuit",
    },
}
twirl_sampler = SamplerV2(backend, options=options)

## Twirling Strategy

active - only twirl gate qubits in each layer
active-accum - twirl the accumulated gate qubits up to and including the current layer
active-circuit - in each layer twirl the union of all gate qubits in the whole circuit
all - in each layer twirl all qubits in the circuit, including idle qubits

## Estimator reilience sub-options

IBM runtime estimator primitives supports built-in error mitigation methods

Enable with the following resilience options
`measure_mitigation` - whether to enable measurement error mitigation method
`zne_mitigation` - whether to turn on Zero noise extrapolation error mitigation method
`pec_mitigation` - whether to turn on probabalistic error cancellation error mitigation method

sub-options:
- `zne` additional options for configuring ZNE mitigation
- `pec` additional options for configuring PEC mitigation
- `measure_noise_learning` additional options for noise learning for measure mitigation


## Estimator resilience levels

built-in resilience levels to automate option configuration
(there is a chart for this which I won't copy here)

In [ ]:
# initialize estimator for ZNE
estimator = EstimatorV2(backend, options={"resilience_level": 2})

# initialize and customize an estimator for ZNE
estimator = EstimatorV2(backend)
estimator.options.resilience_level = 2
estimator.options.dynamical_decoupling.enable = True
estimator.options.dynamical_decoupling.sequence_type = "XY4"
estimator.options.resilience.zne.extrapolator = "linear"